In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_table("data_estag_ds.tsv") # Leitura dos dados

In [3]:
data.head() # Estrutura dos dados

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita


In [4]:
data.isnull().sum() # Nao tem dados faltando

ID       0
TITLE    0
dtype: int64

In [5]:
print("Numero de produtos no total:", len(data))
print("Numero de produtos unicos:", len(data["TITLE"].unique()))

# Não existem muitos produtos repetidos

Numero de produtos no total: 2606
Numero de produtos unicos: 2548


In [6]:
TOKENS = {}
palavras_chave = {}
for i, row in data.iterrows():
    TOKENS[i] = nltk.word_tokenize(str.upper(data["TITLE"][i]))

data["TOKENS"] = TOKENS.values() # Criacao de uma nova feature com os tokens

In [7]:
data.head() # Dataset atualizado

,ID,TITLE,TOKENS
0,1041354,Acessório T - Jean Bag For Girls para DS Lite,"[ACESSÓRIO, T, -, JEAN, BAG, FOR, GIRLS, PARA,..."
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...,"[CARRINHO, DE, BEBÊ, BERÇO-PASSEIO, -, PEGASUS..."
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...,"[CARRINHO, DE, BEBÊ, PARA, GÊMEOS, BERÇO-PASSE..."
3,1042568,Car Center - Calesita,"[CAR, CENTER, -, CALESITA]"
4,1042584,Donka Trem com Som - Calesita,"[DONKA, TREM, COM, SOM, -, CALESITA]"


In [8]:
# Encontra os tokens mais frequentes

palavras_chave = []
for token_list in TOKENS.values():
    for token in token_list:
        palavras_chave.append(token)

palavras_chave = nltk.FreqDist(palavras_chave)
palavras_chave.most_common(10) # 10 Tokens mais comuns

# Apos uma analise dos itens mais comuns, eh possivel identificar objetos que estao associados a smartphones
# porem nao sao smartphones, como: capas, cabos, carregadores, bumpers e outros menos frequentes. 

[('-', 895),
 (',', 770),
 ('CAPA', 650),
 ('GALAXY', 491),
 ('DE', 487),
 ('SAMSUNG', 469),
 ('E', 405),
 ('PARA', 366),
 ('COM', 277),
 ('MOTOROLA', 255)]

In [9]:
# Transforma os tokens em valores numericos

visited_tokens = []
for i, row in data.iterrows():
    token_list = []
    for token in data["TOKENS"][i]:
        if token not in visited_tokens:
            visited_tokens.append(token)
            token_list.append(visited_tokens.index(token))
        else:
            token_list.append(visited_tokens.index(token))
    data["TOKENS"][i] = token_list

In [10]:
# Gera uma matriz de dimensao (numero de produtos, total de tokens) com 0 
# onde nao ha aquele token e 1 onde ha, para cada produto

X = np.zeros((len(data), len(visited_tokens)))
for i, row in data.iterrows():
    for token in data["TOKENS"][i]:
        X[i,token] = 1

In [11]:
# Roda o K means com 36 clusters

kmeans = KMeans(n_clusters=int((len(X)//2)**0.5), random_state=0).fit(X)
data["CATEGORY"] = kmeans.labels_

In [12]:
# Nesta etapa, eu olhei cada cluster manualmente e identifiquei os que se referem a smartphones

smartphones = [0, 6, 7, 8, 9, 14, 15, 16, 18, 19, 28, 32, 34]
data["TITLE"][data["CATEGORY"] == 0].head()

2120    iPhone 8 Plus Apple RED Special Edition 64GB T...
2121    iPhone 8 Plus Apple RED Special Edition 256GB ...
2137    Apple Iphone 5 16gb 100% Original Promoção Imp...
2139    Apple Iphone 5s 16gb Desbloqueado Original Ana...
2143    Apple Iphone 6 16gb Space Gray Original Nacion...
Name: TITLE, dtype: object

In [13]:
# Separa novamente as categorias, porem agora somente em smartphone ou nao-smartphone

for i, row in data.iterrows():
    if data["CATEGORY"][i] in smartphones:
        data["CATEGORY"][i] = 1
    else:
        data["CATEGORY"][i] = 0

In [14]:
# Salva os resultados em formato xlsx (excel) e csv

data[["TITLE","CATEGORY"]].to_excel("resultado.xlsx")
data[["TITLE","CATEGORY"]].to_csv("resultado.csv")

# Análises Finais:
- O número de clusteres no algortimo Kmeans influencia muito na qualidade do agrupamento, portanto, é muito provavel que uma otimização no sentido de encontrar o melhor K possível melhorasse o desempenho do algoritmo. Neste código, foi utilizada uma fórmula padronizada que encontra um K razoável dado o tamanho do dataset floor((n/2)ˆ0.5).<br><br>

- Varios trechos do código deveriam ser otimizados caso o dataset fosse maior. Mesmo com esse tamanho relativamente pequeno, o código todo demora mais do que o necessário para rodar (cerca de 2 minutos).<br><br>

- A abordagem que encontrei para gerar a feature utilizada no treinamento é muito ineficiente em termos de memória, pois gera uma matriz gigantesca. Em datasets maiores, provavelmente outra abordagem deveria ter sido utilizada.<br><br>

- No trecho em que eu olho manualmente cada cluster identificando quais se referem a smartphones, algo deveria ser feito para tornar esse processo automático, pois é inviável realizar esse processo manualmente em datasets maiores. Além disso, caso a seed do treinamento seja alterada, a lista muda completamente e o resultado passa a não fazer mais sentido, já que a lista foi gerada manualmente. Uma solução que me veio à mente seria identificar palavras chaves relacionadas à smartphones e depois agrupar os clusters que possuem várias intâncias com tais palavras.<br><br>

- Eu gostaria de ter extraído mais features do dataset, talvez as distâncias entre os tokens (exemplo: "Celular Smartphone Xiaomi 64gb", distancia("Celular", "Xiaomi") == 2). Achei que o modelo ficou muito simples, mas eu não consegui pensar em uma maneira de padronizar essas features em uma só matriz de input para o algoritmo.<br><br>

- Seria interessante também testar outros algoritmos, mas infelizmente ainda conheço poucos de aprendizado não-supervisionado. Pensei em talvez classificar manualmente algumas instâncias para usar algum algoritmo de aprendizado supervisionado (Naive Bayes, por exemplo), mas me pareceu que o objetivo desse teste era utilizar um algoritmo nao-supervisionado.<br><br>

- Olhando o arquivo resultante, parece que o algoritmo acerta uma taxa razoável de smartphones, mas menos do que eu gostaria. Os erros são mais comuns quando o produto é um smartphone e o modelo o classifica como nao-smartphone. Há poucos ou nenhum caso onde o produto não é um smartphone e foi classificado como smartphone.